In [9]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2
from knockouts import *

In [10]:
MeanTCounts = pandas.read_csv("./ibdmdb/metaT_per_diagnosis.csv",index_col=0)
NormalizedTCounts = MeanTCounts.div(MeanTCounts.sum())
NormalizedTCounts.head()

,nonIBD,UC,CD,IBD
#FeatureID,,,,
HMPREF9436_RS14285,0.078333,0.039315,0.037492,0.037492
HMPREF9436_RS14685,0.023592,0.008788,0.013451,0.013451
BILO145876EF_RS04385,0.022452,0.003468,0.005717,0.005717
FAEPRAM212_RS14865,0.017201,0.023327,0.030562,0.030562
BSFG_RS23450,0.010633,0.011724,0.010224,0.010224


In [11]:
condition = "nonIBD"
# condition = "IBD"
# condition = "UC"
# condition = "CD"

In [12]:
log_format = '%(asctime)s %(message)s' #%(clientip)-15s %(user)-8s
import logging
from coralme.builder.main import ListHandler
log = logging.getLogger()
logging.basicConfig(filename = "./ibdmdb/integration/integration_{}.log".format(condition), filemode = 'w', level = logging.WARNING, format = log_format)
logging.captureWarnings(True)

In [13]:
survivors = set(pandas.read_csv("survivors.txt",index_col=0,header=None).index.to_list())
done = set(i.split("_killable")[0] for i in os.listdir("./ibdmdb/integration/{}".format(condition)) if "txt" in i)
run_for = survivors #- done
len(run_for)

495

In [14]:
def run(org):
    ListHandler.print_and_log("Loading {}".format(org))
    model = coralme.io.pickle.load_pickle_me_model("./me-models/{}/MEModel-BIO-{}-ME-TS.pkl".format(org,org))
    kill_genes = get_targets(model,condition,NormalizedTCounts)
    index_dct = {r.id : model.reactions.index(r) for r in model.reactions}
    pandas.DataFrame(columns=kill_genes).T.to_csv("./ibdmdb/integration_targets/{}/{}_targets.txt".format(condition,org),header=None)
    
    me_nlp = get_nlp(model)
    del model
    
    f,basis = get_feasibility(me_nlp)
    killable = get_killable(index_dct,me_nlp,basis,kill_genes,limit=lambda x:0,ListHandler=ListHandler,org=org)
    
    pandas.DataFrame(columns=killable).T.to_csv("./ibdmdb/integration/{}/{}_killable.txt".format(condition,org),header=None)

In [ ]:
NP = min([2,len(run_for)])
pool = mp.Pool(NP,maxtasksperchild=1)
pbar = tqdm(total=len(run_for),position=0,leave=True)
pbar.set_description('Building ({} threads)'.format(NP))
def collect_result(result):
    pbar.update(1)
    
for org in run_for:
    args = ([org])
    pool.apply_async(run,args, callback=collect_result)
pool.close()
pool.join()

Building (2 threads):   0%|          | 0/45 [00:00<?, ?it/s]

Loading Lactobacillus_paracasei_subsp_paracasei_8700_2Loading Streptococcus_mutans_ATCC_25175

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmp7rmlj14r.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmpwux8t01v.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmpih34co7s.lp
Reading time = 0.00 seconds
: 1070 rows, 2538 columns, 10864 nonzeros
Read LP format model from file /tmp/tmp759b6a87.lp
Reading time = 0.00 seconds
: 953 rows, 2058 columns, 8792 nonzeros
Read LP format model from file /tmp/tmpze4oui4v.lp
Reading time = 0.00 seconds
: 1087 rows, 2536 columns, 10664 nonzeros
Read LP format model from file /tmp/tmpgc7kboej.lp
Reading time = 0.00 seconds
: 976 rows, 2056 columns, 8672 nonzeros
Lactobacillus_paracas

Building (2 threads):   2%|▏         | 1/45 [18:32<13:35:52, 1112.55s/it]

Loading Varibaculum_cambriense_DSM_15806
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmpc5frn23u.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmphs272wnc.lp
Reading time = 0.00 seconds
: 940 rows, 2026 columns, 8774 nonzeros
Read LP format model from file /tmp/tmpr_8bpt5d.lp
Reading time = 0.00 seconds
: 941 rows, 2024 columns, 8604 nonzeros
Lactobacillus_paracasei_subsp_paracasei_8700_2 feasible if 537973.8.peg.1382 knockout (401 out of 846)
Lactobacillus_paracasei_subsp_paracasei_8700_2 feasible if 537973.8.peg.1383 knockout (402 out of 846)
Lactobacillus_paracasei_subsp_paracasei_8700_2 feasible if 537973.8.peg.1384 knockout (403 out of 846)
Lactobacillus_paracasei_subsp_paracasei_8700_2 feasible if 537973.8.peg.1387 knockout (404 out of 846)
Lactobacillus_paracasei_subsp_paracasei_8700_2 feasible if 537973.8.peg.1388 knockout (405 out of 846)
Lact

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Bacillus_fordii_DSM_16014 not feasible if g.29556.CDS.570 knockout (161 out of 1237)
Staphylococcus_simulans_ACS_120_V_Sch1 not feasible if g.28050.CDS.2048 knockout (750 out of 962)
Bacillus_fordii_DSM_16014 feasible if g.29556.CDS.572 knockout (162 out of 1237)
Staphylococcus_simulans_ACS_120_V_Sch1 feasible if g.28050.CDS.2053 knockout (751 out of 962)
Staphylococcus_simulans_ACS_120_V_Sch1 not feasible if g.28050.CDS.2054 knockout (752 out of 962)
Bacillus_fordii_DSM_16014 not feasible if g.29556.CDS.573 knockout (163 out of 1237)
Staphylococcus_simulans_ACS_120_V_Sch1 not feasible if g.28050.CDS.2055 knockout (753 out of 962)
Staphylococcus_simulans_ACS_120_V_Sch1 feasible if g.28050.CDS.2057 knockout (754 out of 962)
Bacillus_fordii_DSM_16014 feasible if g.29556.CDS.578 knockout (164 out of 1237)
Staphylococcus_simulans_ACS_120_V_Sch1 feasible if g.28050.CDS.2060 knockout (755 out of 962)
Bacillus_fordii_DSM_16014 not feasible if g.29556.CDS.583 knockout (165 out of 1237)
Staphyl

Building (2 threads):  18%|█▊        | 8/45 [1:49:43<9:41:22, 942.76s/it]

Loading Campylobacter_jejuni_jejuni_81_176
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmpoia5lfq6.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmpakifqxb6.lp
Reading time = 0.00 seconds
: 942 rows, 2132 columns, 9312 nonzeros
Bacillus_fordii_DSM_16014 feasible if g.29556.CDS.842 knockout (239 out of 1237)
Read LP format model from file /tmp/tmpu8d93y2n.lp
Reading time = 0.00 seconds
: 970 rows, 2130 columns, 9142 nonzeros
Bacillus_fordii_DSM_16014 feasible if g.29556.CDS.844 knockout (240 out of 1237)
Bacillus_fordii_DSM_16014 not feasible if g.29556.CDS.849 knockout (241 out of 1237)
Campylobacter_jejuni_jejuni_81_176 feasible if 6x7.58933.peg.14 knockout (1 out of 574)
Campylobacter_jejuni_jejuni_81_176 feasible if 6x7.58933.peg.53 knockout (2 out of 574)
Campylobacter_jejuni_jejuni_81_176 feasible if 6x7.58933.peg.54 knockout (3 out of 574)
Campyl

Building (2 threads):  20%|██        | 9/45 [2:00:22<8:28:41, 847.82s/it]

Loading Bacillus_cereus_AH187_F4810_72
Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16
Read LP format model from file /tmp/tmpavlplqo5.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmpqf0p12bo.lp
Reading time = 0.00 seconds
: 1265 rows, 3146 columns, 13546 nonzeros
Read LP format model from file /tmp/tmp0dhrduci.lp
Reading time = 0.00 seconds
: 1290 rows, 3144 columns, 13346 nonzeros
Bacillus_fordii_DSM_16014 feasible if g.29556.CDS.1169 knockout (328 out of 1237)
Bacillus_fordii_DSM_16014 feasible if 1121090.3.peg.1173 knockout (329 out of 1237)
Bacillus_fordii_DSM_16014 feasible if 1121090.3.peg.1174 knockout (330 out of 1237)
Bacillus_fordii_DSM_16014 feasible if g.29556.CDS.1179 knockout (331 out of 1237)
Bacillus_fordii_DSM_16014 feasible if g.29556.CDS.1184 knockout (332 out of 1237)
Bacillus_fordii_DSM_16014 feasible if g.29556.CDS.1185 knockout (333 out of 1237)
Bacillus_fordii_DSM_1

In [31]:
l1 = set("RNA_"+i for i in pandas.read_csv("./ibdmdb/integration/nonIBD/Abiotrophia_defectiva_ATCC_49176_killable.txt",index_col=0).index)
l2 = set(pandas.read_csv("./ibdmdb/integration/nonIBD/_Abiotrophia_defectiva_ATCC_49176_killable.txt",index_col=0).index)
l3 = set("RNA_"+i for i in pandas.read_csv("./ibdmdb/reduction/nonIBD/Abiotrophia_defectiva_ATCC_49176_killable.txt",index_col=0).index)